In [2]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [88]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 1,
                                   22: 3, 23: 2, 24: 2, 25: 5, 26: 3,
                                   27: 7, 28: 4, 29: 22, 30: 12,
                                   31: 14, 32: 23, 33: 11, 34: 24,
                                   35: 18, 36: 31, 37: 19, 38: 22,
                                   39: 34, 40: 22, 41: 29, 42: 29,
                                   43: 31, 44: 26, 45: 29, 46: 25,
                                   47: 12})

In [89]:
chi_last_day = max(chi_death_dict.keys())

In [90]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [91]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [92]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-04-12,31
2020-04-13,26
2020-04-14,29
2020-04-15,25
2020-04-16,12


In [93]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')

### Illinois
Data source for Illinois:

http://www.dph.illinois.gov/covid19/covid19-statistics

`il_cum_death_list` is the list of cumulative deaths on each day, starting on March 16. Again, save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [94]:
il_cum_death_list = [0, 1, 3, 4, 5, 6, 9, 12, 16, 19, 26, 34, 47,
                     65, 72, 99, 141, 157, 210, 243, 274, 307, 380,
                     462, 528, 596, 677, 720, 794, 868, 948, 1072,
                     1134]

In [95]:
# New deaths from March 17
il_death_list = [now - prev 
                 for now, prev in zip(il_cum_death_list[1:],
                                      il_cum_death_list[:-1])]

In [96]:
il_death_dict = defaultdict(int, {index + 17: diff
                                  for index, diff in enumerate(il_death_list)})

In [97]:
il_num_days = len(il_death_list)

In [98]:
num_days = len(il_death_list)

In [99]:
il_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + 16 + num_days)]
il_deaths =[il_death_dict[day - 28] for day in range(29 + 16 + num_days)]

In [100]:
il_death_df = pd.DataFrame(il_deaths, index=il_index, columns=['deaths'])

In [101]:
# Inspect that last few days
il_death_df.tail()

,deaths
2020-04-13,74
2020-04-14,74
2020-04-15,80
2020-04-16,124
2020-04-17,62


In [102]:
il_death_df.to_pickle('../data/il_deaths.pickle')